### Dataload and preprocessing

In [64]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, roc_curve, roc_auc_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import precision_recall_fscore_support

from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [65]:
acc_hr_user01_06_path = r'C:\Users\user\Desktop\work\AI_study\korea_univ_std\ETRI_human\test_code\my_test_model\acc_hr_user01_06'

acc_hr_user01_06_df = pd.DataFrame()

# 디렉토리의 모든 파일을 순회하며 DataFrame에 추가
for file in os.listdir(acc_hr_user01_06_path):
    file_path = os.path.join(acc_hr_user01_06_path, file)
    df = pd.read_csv(file_path)
    acc_hr_user01_06_df = pd.concat([acc_hr_user01_06_df, df], axis=0)

In [66]:
# 결측치 제거
acc_hr_user01_06_df = acc_hr_user01_06_df.dropna()

# drop column
acc_hr_user01_06_df.drop(columns=['Unnamed: 0.1','Unnamed: 0','subject_id'],inplace=True)

# datetime set
acc_hr_user01_06_df['date'] = pd.to_datetime(acc_hr_user01_06_df['date'])
acc_hr_user01_06_df['date'] = (acc_hr_user01_06_df['date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
                                                       

In [67]:
acc_hr_user01_06_df_sds = pd.DataFrame()
scaler = StandardScaler()

acc_hr_user01_06_df_sds[list(acc_hr_user01_06_df.columns[:30])] = scaler.fit_transform(acc_hr_user01_06_df.iloc[:,:30])

In [68]:
y = acc_hr_user01_06_df.iloc[:,31:]

X_train, X_test, y_train, y_test = train_test_split(acc_hr_user01_06_df_sds, y)

In [74]:
# 데이터를 텐서로 변환
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# 데이터 로더 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32)  

### Model Training

In [77]:
class MultiLabelNN(nn.Module):
    def __init__(self):
        super(MultiLabelNN, self).__init__()

        self.layer1 = nn.Linear(X_train.shape[1], 64)
        self.relu1 = nn.ReLU()

        self.layer2 = nn.Linear(64, 128)
        self.relu2 = nn.ReLU()
        
        self.layer3 = nn.Linear(128, 128)
        self.relu3 = nn.ReLU()
        
        self.layer4 = nn.Linear(128, 64)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(0.1)
        
        self.output = nn.Linear(64, y_train.shape[1])
        
    
    def forward(self, x):
        x = self.relu1(self.layer1(x))
        x = self.relu2(self.layer2(x))
        x = self.relu3(self.layer3(x))
        x = self.dropout4(self.relu4(self.layer4(x)))
        x = torch.sigmoid(self.output(x))
        return x


In [80]:
model = MultiLabelNN()

criterion = nn.BCELoss()  # binary cross entropy
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# 훈련 루프
def train_model(model, train_loader, criterion, optimizer, epochs=600):
    model.train()
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train_model(model, train_loader, criterion, optimizer)

Epoch 1, Loss: 0.6881439685821533
Epoch 2, Loss: 0.6855859756469727
Epoch 3, Loss: 0.6925676465034485
Epoch 4, Loss: 0.7041584849357605
Epoch 5, Loss: 0.6552127003669739
Epoch 6, Loss: 0.6087868213653564
Epoch 7, Loss: 0.5735003352165222
Epoch 8, Loss: 0.630150556564331
Epoch 9, Loss: 0.6339369416236877
Epoch 10, Loss: 0.6107613444328308
Epoch 11, Loss: 0.5932239890098572
Epoch 12, Loss: 0.6608797907829285
Epoch 13, Loss: 0.5805125832557678
Epoch 14, Loss: 0.6061140298843384
Epoch 15, Loss: 0.45783472061157227
Epoch 16, Loss: 0.5007861256599426
Epoch 17, Loss: 0.4715280830860138
Epoch 18, Loss: 0.6006022095680237
Epoch 19, Loss: 0.5645208358764648
Epoch 20, Loss: 0.48507484793663025
Epoch 21, Loss: 0.4564422369003296
Epoch 22, Loss: 0.4738701283931732
Epoch 23, Loss: 0.4898134469985962
Epoch 24, Loss: 0.4111727476119995
Epoch 25, Loss: 0.474274218082428
Epoch 26, Loss: 0.3912581205368042
Epoch 27, Loss: 0.34392619132995605
Epoch 28, Loss: 0.3849621117115021
Epoch 29, Loss: 0.2670250236

### model evaluation

In [81]:
def evaluate_model(model, test_loader):
    model.eval()  # 평가 모드로 설정
    total_loss = 0
    all_predictions = []
    all_labels = []
    criterion = nn.BCELoss()  # 이진 크로스 엔트로피 손실
    
    with torch.no_grad():  # 그래디언트 계산 비활성화
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            # 예측값을 0 또는 1로 변환
            predicted = outputs > 0.5
            all_predictions.append(predicted.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            
    print(all_predictions)

    all_predictions = np.vstack(all_predictions)
    all_labels = np.vstack(all_labels)
    
    # 스코어 계산
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='macro')
    
    average_loss = total_loss / len(test_loader)
    print(f'Average Loss: {average_loss:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

# 모델 평가
evaluate_model(model, test_loader)

[array([[ True,  True, False, False,  True,  True,  True],
       [False, False, False, False,  True,  True,  True],
       [ True,  True, False, False,  True,  True,  True],
       [ True, False, False, False, False, False,  True],
       [False, False, False, False,  True, False,  True],
       [ True,  True, False, False, False, False, False],
       [False, False,  True,  True,  True, False,  True],
       [False,  True,  True, False, False, False,  True],
       [ True,  True,  True,  True, False, False, False],
       [False,  True, False, False,  True,  True,  True],
       [False, False, False,  True,  True,  True,  True],
       [False,  True,  True,  True,  True, False,  True],
       [False, False, False, False,  True,  True,  True],
       [False, False, False,  True,  True,  True,  True],
       [False,  True,  True,  True,  True,  True,  True],
       [False,  True, False, False,  True,  True,  True],
       [False,  True,  True, False, False, False, False],
       [False